In part 3 we'll standartize our residuals to catch uncatched variance.

We'll evaluate performance of our models using Score calculation described in part 1, using TimeSeriesSplit cross-validation. However, we'll use a new function, ['functions/get_score_3.py'](functions/get_score_3.py). 

This function will predict scaled residuals ('emission_03_scaled') and then add to descaled predictions Trend and Seasonality to predict 'emission'. Then it will calculate the Score, in the same way as in part 1.

I tested (testing is not included here) calculated in part 2 'Cycles' feature, but this way gave me lower Cross-val score with TimeSeriesSplit cross-validation, so I won't use 'Cycles'.

Let's load the datasets:

In [1]:
%%time

import pandas as pd
import numpy as np


global_variables = pd.read_csv('global_variables.csv', index_col=0)
SEED = global_variables.loc[0, 'SEED']
N_SPLITS = global_variables.loc[0, 'N_SPLITS']

train_unprocessed = pd.read_csv('datasets/train.csv', index_col='ID_LAT_LON_YEAR_WEEK')
test_unprocessed = pd.read_csv('datasets/test.csv', index_col='ID_LAT_LON_YEAR_WEEK')
train_from_part_1 = pd.read_csv('new_datasets/train_from_part_1.csv', index_col='ID_LAT_LON_YEAR_WEEK')
test_from_part_1 = pd.read_csv('new_datasets/test_from_part_1.csv', index_col='ID_LAT_LON_YEAR_WEEK')
train_from_part_2 = pd.read_csv('new_datasets/train_from_part_2.csv', index_col='ID_LAT_LON_YEAR_WEEK')
test_from_part_2 = pd.read_csv('new_datasets/test_from_part_2.csv', index_col='ID_LAT_LON_YEAR_WEEK')

top_three_values = train_from_part_1.loc[:, 'Location_enc'].drop_duplicates().sort_values(ascending = False).head(3)
top_three_locations = train_from_part_1.loc[train_from_part_1['Location_enc'].isin(top_three_values), 'Location'].drop_duplicates()


CPU times: total: 2.16 s
Wall time: 2.69 s


We'll create a DataFrame that won't be used in models training, but just for calculating the final prediction:

In [2]:
post_training_df = pd.concat([train_from_part_2, test_from_part_2])[['Trend', 'Seasonality', 'emission']]

Now, let's create 'train' and 'test' sets, that will contain all the variables we may use for our predictions, making sure we won't include variables that caclulate 'emission' from residuals, to avoid data leakage:

In [3]:
train = pd.concat([train_unprocessed.drop('emission', axis=1), 
                   train_from_part_2[['Location', 'WeekCount', 'date']]], axis=1)
train['date'] = pd.to_datetime(train['date'])
test = pd.concat([test_unprocessed, test_from_part_2[['Location', 'WeekCount', 'date']]], axis=1)
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

Let's standartize the residuals:

In [4]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns

scaler = StandardScaler()

train['emission_03_scaled'] = scaler.fit_transform(train_from_part_2[['emission_03']])

In [5]:
x = np.array([[2,3]])

x[0]


array([2, 3])

## 00. Baseline.

As a baseline, we'll fit LightGBM estimator to one feature, which will be just the column of 0's. We still should get quite a good Score (I remind you that lower is merrier), because predicted residuals will be transformed into 'emission' using features we calculated in Part 2:

In [6]:
%%time

study_number = '00'

scores_df = pd.DataFrame({'Comment': [], 'Train Score': [], 'Cross-val Score': [], 'Test RMSE': []})

train_selected = pd.DataFrame(index=train.index, columns=['dummy', 'emission_03_scaled'])
train_selected['dummy'] = 0
train_selected['emission_03_scaled'] = train['emission_03_scaled']
test_selected = pd.DataFrame(index=test_unprocessed.index, columns=['dummy'])
test_selected['dummy'] = 0


# Instantiate the estimator
# UNCOMMENT TO INSTALL LightGBM
#!pip install lightgbm
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=SEED, n_jobs=-1, n_estimators=10)

# Calculate scores
from functions.get_score_3 import get_score_3

train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     scaler,
                                                                     post_training_df, model, scores_df,
                                                                  comment="Part 3 Baseline")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

[[-0.10200653]
 [-0.10200653]
 [-0.10200653]
 ...
 [-0.10200653]
 [-0.10200653]
 [-0.10200653]]
[-0.10200653]


ValueError: could not broadcast input array from shape (15807,) into shape (1,)

In [7]:
scores_df.loc[int(study_number), 'Test RMSE'] = 29.32305
scores_df

,Comment,Train Score,Cross-val Score,Test RMSE
0,NaN,NaN,NaN,29.32305


## 01. Location Median Target Encoding

Next, we'll recalculate our 'Location_enc' feature, same as Part 1, but with respect to residuals:

In [8]:
study_number = '01'

location_median = train.groupby('Location')['emission_03'].median()

train['Location_enc'] = train['Location'].map(location_median)
test['Location_enc'] = test['Location'].map(location_median)

feature_name = 'Location_enc'
for data in [train, test]:
    print(data[feature_name].info())
    print(data[feature_name].describe())
    print('Unique Values and their count:')
    print(data[feature_name].value_counts())

KeyError: 'Column not found: emission_03'

Let's test:

In [ ]:
%%time

selected_columns = ['Location_enc']

train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]

#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="Location_enc")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

We don't see an improvement, so we won't use this feature.

In [ ]:
selected_columns.remove('Location_enc')

## 02. Week of the year Median Target Encoding

What if we do encoding on the 'week_no' feature? Potentially, it may catch some weekly common sesoanalities, if they exist.

In [ ]:
study_number = '02'

week_no_median = train.groupby('week_no')['emission_03'].median()

train['week_no_enc'] = train['week_no'].map(week_no_median)
test['week_no_enc'] = test['week_no'].map(week_no_median)

feature_name = 'week_no_enc'
for data in [train, test]:
    print(data[feature_name].info())
    print(data[feature_name].describe())
    print('Unique Values and their count:')
    print(data[feature_name].value_counts())


Let's test:

In [ ]:
%%time

selected_columns.append('week_no_enc')

train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]


#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="week_no_enc")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

No, that didn't work, maybe, there is no common seasonality left across all Locations.

In [ ]:
selected_columns.remove('week_no_enc')

train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]

## 03. Most important features

Let's estimate what features are most important for calculating our residues:

In [ ]:
%%time

study_number = '03'

## Train the model on the whole training set
model.fit(train.drop(['emission_03', 'Location', 'date'], axis=1), train['emission_03'])

feature_importances = pd.Series(data=model.feature_importances_, 
                                index=train.drop(['emission_03', 'Location', 'date'], axis=1).columns)

print('Sorted feature importances (first 40):')
feature_importances.sort_values(ascending=False).head(40)

Few observations:

-) The same model used way more features in calculating residues, than in calculating 'emission' (see Part 1).

-) In combination with other features, 'Location_enc' and'week_no_enc' were useful for the model (although, they may still not pass the cross-validation testing)
-) Even though we have 'Location_enc' and 'week_no_enc', week_no, latitudes and longitudes are still important

Let's test a model based on some of the most important features:

In [ ]:
%%time

selected_columns = list(feature_importances.sort_values(ascending=False).index[:15])

print('Selected columns: {}'.format(selected_columns))
train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]


#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="Most important features")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

## 04. Geographical clustering

The importance of latitude and longitude suggest benefits of creating location clusters: groups of locations that situated nearby. We'll also account for the 'emissions_03' to make clusters depending on similar emissions.

First we'll create an elbow plot to detrmine an optimal number of clusters:

In [ ]:
%%time

study_number = '04'

from sklearn.preprocessing import StandardScaler
from scipy.cluster.vq import kmeans, vq
import matplotlib.pyplot as plt
import seaborn as sns


# Create cluster data
cluster_data = train[['latitude', 'longitude', 'emission_03']]

# Normalize data
cluster_data = pd.DataFrame(StandardScaler().fit_transform(cluster_data), columns=['latitude', 'longitude', 'emission_03'],
                           index=cluster_data.index)

distortions = []
num_clusters = range(1, 7)

# Create a list of distortions from the kmeans function
for i in num_clusters:
    cluster_centers, distortion = kmeans(cluster_data, k_or_guess=i, seed=SEED)
    distortions.append(distortion)

# Create a DataFrame with two lists - num_clusters, distortions
elbow_plot = pd.DataFrame({'num_clusters': num_clusters, 'distortions': distortions})

# Creat a line plot of num_clusters and distortions
sns.lineplot(x='num_clusters', y='distortions', data = elbow_plot)
plt.xticks(num_clusters)
plt.show()

It seems that the optimal number of clusters is 2, but 4 can be also an option. Let's start with 2 clusters and look how they are situaded geographically:

In [ ]:
%%time

cluster_centers, distortion = kmeans(cluster_data, k_or_guess=2, seed=SEED)


# Assign cluster labels
cluster_data['cluster_labels'], _ = vq(cluster_data, cluster_centers)


# Create a scatter plot
sns.scatterplot(x='latitude', y='longitude', hue='cluster_labels', data=cluster_data)
plt.show()

Note, that even though we used scaled 'emission_03' in our clustering, our clusters have nice geographical borders. Now, let's test:

In [ ]:
%%time

# Save the cluster labels into train and set
clusters = pd.concat([train['Location'], cluster_data['cluster_labels']], axis=1)
clusters = pd.pivot_table(clusters, values=['cluster_labels'], index=['Location'])
clusters = pd.Series(clusters['cluster_labels'])

train['Location_cluster_2'] = train['Location'].map(clusters)
test['Location_cluster_2'] = test['Location'].map(clusters)

selected_columns = ['Location_cluster_2']

print('Selected columns: {}'.format(selected_columns))
train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]

#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="Location_cluster_2")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

That didn't work.

## 05. Geographical clustering (4 clusters)

Let's try 4 clusters:

In [ ]:
%%time

study_number = '05'

selected_columns.remove('Location_cluster_2')

cluster_centers, distortion = kmeans(cluster_data, k_or_guess=4, seed=SEED)


# Assign cluster labels
cluster_data['cluster_labels'], _ = vq(cluster_data, cluster_centers)


# Create a scatter plot
sns.scatterplot(x='latitude', y='longitude', hue='cluster_labels', data=cluster_data)
plt.show()

In [ ]:
%%time

# Save the cluster labels into train and set
clusters = pd.concat([train['Location'], cluster_data['cluster_labels']], axis=1)
clusters = pd.pivot_table(clusters, values=['cluster_labels'], index=['Location'])
clusters = pd.Series(clusters['cluster_labels'])

train['Location_cluster_4'] = train['Location'].map(clusters)
test['Location_cluster_4'] = test['Location'].map(clusters)

selected_columns = ['Location_cluster_4']

print('Selected columns: {}'.format(selected_columns))
train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]

#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected, test_selected, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="Location_cluster_4")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

That didn't work.

To be sure, let's expore how our new features affect feature importances:

In [ ]:
## Train the model on the whole training set
model.fit(train.drop(['emission_03', 'Location', 'date'], axis=1), train['emission_03'])

feature_importances = pd.Series(data=model.feature_importances_, 
                                index=train.drop(['emission_03', 'Location', 'date'], axis=1).columns)

print('Sorted feature importances (first 40):')
feature_importances.sort_values(ascending=False).head(40)

## 06. Filling NaNs based on 15 features

First, we'll use KNNImputer to input missing values based on 15 most important features:

In [ ]:
%%time

study_number = '06'

selected_columns = list(feature_importances.sort_values(ascending=False).index[:15])

print('Selected columns: {}'.format(selected_columns))
train_selected = train[selected_columns]
train_selected = pd.concat([train_selected, train['emission_03']], axis=1)
test_selected =  test[selected_columns]

from sklearn.impute import KNNImputer

# define imputer
imputer = KNNImputer(weights='distance')
# fit on the dataset
imputer.fit(train_selected.drop('emission_03', axis=1))
# transform the datasets
Xtrans = imputer.transform(train_selected.drop('emission_03', axis=1))
train_selected_filled = pd.DataFrame(Xtrans, index=train_selected.index)
train_selected_filled = pd.concat([train_selected_filled, train_selected['emission_03']], axis=1)
train_selected_filled.columns = train_selected.columns
Xtrans = imputer.transform(test_selected)
test_selected_filled = pd.DataFrame(Xtrans, index=test_selected.index, columns=test_selected.columns)


In [ ]:
train_selected_filled.info()

In [ ]:
#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected_filled, 
                                                                     test_selected_filled, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="kNN imputing on 15")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

This filling works worse than NaNs.

## 07. Filling NaNs based on 5 most important

Now we'll use 5 most important features in calculating distances. They contain information about location and week of the year.

In [ ]:
%%time

study_number = '07'

selected_columns_5 = list(feature_importances.sort_values(ascending=False).index[:5])
five_features_train = train[selected_columns_5]
five_features_test =  test[selected_columns_5]

# For each column with missing values
for column in list(train_selected.columns[train_selected.isna().any()]):
    
    # Create a DataFrame of a feature with missing values and 5 most important features
    df_to_fill_train = pd.concat([train[column], five_features_train], axis=1)
    df_to_fill_test = pd.concat([test[column], five_features_test], axis=1)
                                  
    # fit on the dataset
    imputer.fit(df_to_fill_train)
    # transform the datasets
    Xtrans = imputer.transform(df_to_fill_train)
    train_selected_filled[column] = Xtrans[:, 0]
    Xtrans = imputer.transform(df_to_fill_test)
    test_selected_filled[column] = Xtrans[:, 0]



In [ ]:
train_selected_filled.info()

In [ ]:
test_selected_filled.info()

In [ ]:
#Calculate scores
train_score, cross_score, cross_scores_std, submission = get_score_3(global_variables, train_selected_filled,
                                                                     test_selected_filled, 
                                                                     post_training_df, model, scores_df,
                                                                  comment="kNN imputing based on 5")

submission.to_csv('submissions/submission_3_' + study_number + '.csv', index=False)

In [ ]:
scores_df.loc[int(study_number), 'Test RMSE'] = np.nan
scores_df

As we can see, kNN imputing didn't work. 